In [ ]:
import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(""))))
from CausalAbstraction.tasks.two_digit_addition_task.arithmetic import get_token_positions, get_task

NameError: name '__file__' is not defined

In [ ]:
task = get_task(hf=True,size=10000)
print("Raw input:")
print(task.raw_all_data["input"][0])
task.display_counterfactual_data() 

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
from CausalAbstraction.pipeline import LMPipeline
import torch

device = "cuda:1" if torch.cuda.is_available() else "cpu"

# model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
# model_name = "Qwen/Qwen2.5-0.5B"
model_name = "google/gemma-2-2b"

if "llama" in model_name:
    max_new_tokens = 1
elif "gemma" in model_name:
    max_new_tokens = 3

def checker(output_text, expected):
    if expected[0] == "0":
        expected = expected[1:]
        if max_new_tokens == 3:
            return output_text[-3:-1] == expected
    return expected == output_text


pipeline = LMPipeline(model_name, max_new_tokens=max_new_tokens, device=device, dtype=torch.float16)
pipeline.tokenizer.padding_side = "left"
batch_size = 1024
print("DEVICE:", pipeline.model.device)

print("INPUT:", task.raw_all_data["input"][0])
print("LABEL:", task.raw_all_data["label"][0])
print("PREDICTION:", pipeline.dump(pipeline.generate(task.raw_all_data["input"][0])))

task.filter(pipeline, checker, verbose=True, batch_size=batch_size)

In [ ]:

token_positions = get_token_positions(pipeline, task)
input = task.sample_raw_input()
print(input)
for token_position in token_positions:
    print(token_position.highlight_selected_token(input))

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

start = 0
end = pipeline.get_num_layers()
config={"batch_size":256 , "evaluation_batch_size": batch_size, "training_epoch":1, "n_features":16, "regularization_coefficient":0.0, "output_scores":False}
names = ["random", "ones_carry"]
train_data = [name + "_train" for name in names]
validation_data = [name + "_validation" for name in names]
test_data = [name + "_test" for name in names]
# test_data += [name + "_testprivate" for name in names]
verbose = True
results_dir = "math_results"

In [ ]:
from CausalAbstraction.experiments.aggregate_experiments import residual_stream_baselines
residual_stream_baselines(pipeline=pipeline, task=task, token_positions=token_positions, train_data=train_data, test_data=test_data, config=config, target_variables=["ones_carry"], checker=checker, start=start, end=end, verbose=verbose, results_dir=results_dir, 
                          methods=["DBM+SAE"])